In [1]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import pickle as pkl

### 处理缺失值

In [2]:
#填充缺失值,考虑首尾缺失的可能
def fill_missing(df,day=1192):
    '''
    存在每个brand开头缺失和结尾缺失的情况
    '''
    modi_df = pd.DataFrame(columns=['date', 'day_of_week', 'cnt'], dtype=int)
    last_day_of_week = 0
    for index, row in df.iterrows():
        now_day_of_week = row['day_of_week']
        # 若 day_of_week 当前天减上一天不是1 或 -6，说明有跳跃
        while (last_day_of_week != 0) and (now_day_of_week - last_day_of_week != 1) and (now_day_of_week - last_day_of_week != -6):
            if last_day_of_week == 7:
                last_day_of_week = 1
            else:
                last_day_of_week += 1
            modi_df = modi_df.append({'day_of_week': last_day_of_week, 'cnt': 0}, ignore_index=True) 
        modi_df = modi_df.append(row, ignore_index=True)
        last_day_of_week = now_day_of_week
    df_len = modi_df.shape[0]
    if df_len != day:
        #print df_len
        #开头部分有缺失
        if modi_df.iloc[0]['day_of_week'] != 3:
            now_day_of_week = modi_df.iloc[0]['day_of_week']
            last_day_of_week = 3
            above = pd.DataFrame([[last_day_of_week,0,np.nan]],columns=['day_of_week','cnt','date'])
            last_day_of_week += 1
            while last_day_of_week != now_day_of_week:
                above = above.append({'day_of_week': last_day_of_week, 'cnt': 0}, ignore_index=True)
                if last_day_of_week == 7:
                    last_day_of_week = 1
                else:
                    last_day_of_week += 1
            modi_df = above.append(modi_df)
        df_len = modi_df.shape[0]     
        assert (modi_df.iloc[0]['day_of_week'] == 3)
        #结尾缺失
        #if (day- df_len)%7 == 0 or modi_df.iloc[0]['day_of_week'] == 3:
        if df_len != day:
            day_num = day-df_len
            #print 'day_num:',day_num
            #填充滿
            last_day_of_week = modi_df.iloc[-1]['day_of_week']
            while (day_num>0):
                if last_day_of_week == 7:
                    last_day_of_week =1
                else:
                    last_day_of_week += 1
                day_num -= 1
                modi_df = modi_df.append({'day_of_week': last_day_of_week, 'cnt': 0}, ignore_index=True)
                #print 'modi_df.shape:',modi_df.shape         
    return modi_df

In [3]:
#处理缺失值
def fill_missing2(df):
    modi_df = pd.DataFrame(columns=['date', 'day_of_week', 'cnt'], dtype=int)
    last_day_of_week = 0
    for index, row in df.iterrows():
        now_day_of_week = row['day_of_week']
        # 若 day_of_week 当前天减上一天不是1 或 -6，说明有跳跃
        while (last_day_of_week != 0) and (now_day_of_week - last_day_of_week != 1) and (now_day_of_week - last_day_of_week != -6):
            if last_day_of_week == 7:
                last_day_of_week = 1
            else:
                last_day_of_week += 1
            modi_df = modi_df.append({'day_of_week': last_day_of_week, 'cnt': 0}, ignore_index=True)
        modi_df = modi_df.append(row, ignore_index=True)
        last_day_of_week = now_day_of_week
    return modi_df

### 构建预测模型

In [4]:
 #确定预测集大小
test_A = pd.read_csv('./data/test_A_20171225.txt',sep='\t')
test_B = pd.read_csv('./data/test_B_20171225.txt',sep='\t')
all_test = pd.concat([test_A,test_B])
modi_test_A = fill_missing2(all_test)
modi_test_A.cnt = pd.to_numeric(modi_test_A.cnt)
print modi_test_A.shape
modi_test_A['ds'] = pd.date_range(start = '4/7/2016', periods=601)

(601, 3)


In [5]:
#针对每个品牌分别使用prophet建模并预测最后合并预测结果
def run_based_brand(brand,d):
    #brand:品牌
    #d:brand对应的数据
    train = d
    new_df = train.drop(['day_of_week', 'brand'],axis=1).groupby('date').sum()
    new_df['date'] = new_df.index
    new_df = pd.merge(new_df, train[['date','day_of_week']].drop_duplicates(), how='left', on='date')
    #print new_df.shape
    #new_df.to_csv('./data/train_group.txt', sep='\t')
    #填充缺失值
    modi_df = fill_missing(new_df)
    #print '填充缺失值：',modi_df.shape
    # 改变 cnt 列的值属性为数值型
    modi_df.cnt = pd.to_numeric(modi_df.cnt)
    
    #均值按星期填充
    for i in range(1, 8):
        # 对每个 day_of_week 依次填充
        day_of_week_mean = int(new_df[new_df.day_of_week == i]['cnt'].mean())
        modi_df.loc[modi_df.day_of_week == i, 'cnt']= modi_df[modi_df.day_of_week == i].cnt.mask(modi_df.cnt == 0, day_of_week_mean)
    #print '按星期均值填充：',modi_df.shape 
    
    #为现有的dataframe增加ds
    modi_df['ds'] = pd.date_range(start = '1/2/2013', periods=1192)
    prophet_train = modi_df.loc[:,['ds', 'cnt']]
    prophet_train.rename(columns = {'cnt':'y'}, inplace = True)
    
    #导入假期数据
    with open('./data/holidays.pkl', 'rb') as rf:
        holidays = pkl.load(rf)
        
    #训练模型
    m = Prophet(holidays = holidays)
    m.fit(prophet_train)
   
    #预测
    future = pd.DataFrame({'ds':pd.date_range(start='4/8/2016',periods=600)})
    forecast = m.predict(future)
    test_A_yhat = forecast.loc[:, ['ds', 'yhat']]
    test_A_yhat = test_A_yhat.merge(modi_test_A, on=['ds'])
    real_pre_yhat_A = test_A_yhat[test_A_yhat.date.notnull()].drop(['day_of_week', 'cnt', 'ds'],axis=1)
    print real_pre_yhat_A.shape
    real_pre_yhat_A[['yhat', 'date']] = real_pre_yhat_A[['yhat', 'date']].astype(int)
    return real_pre_yhat_A

In [6]:
real_pre_list = []
df = pd.read_csv('./data/train_20171215.txt',sep='\t')
df_group = df.groupby('brand')
for num,d in df_group:
    real_pre_yhat = run_based_brand(num,d)
    real_pre_list.append(real_pre_yhat)


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


(547, 2)


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


(547, 2)


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


(547, 2)


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


(547, 2)


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


(547, 2)


In [7]:
result = pd.DataFrame(columns=['date','yhat'])
result['date'] = real_pre_list[0].date
result['yhat'] = sum([x.yhat for x in real_pre_list])

### 存储预测结果

In [8]:
result.loc[:,['date','yhat']].to_csv('./data/final_test_B_result.txt',sep='\t',index=False)